In [25]:
import pandas as pd
import datetime
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings('ignore')
import math

In [26]:
royal=pd.read_excel('trips data.xlsx',sheet_name='royal')
volcano=pd.read_excel('trips data.xlsx',sheet_name='volcano')
kbs=pd.read_excel('trips data.xlsx',sheet_name='kbs')

royal,volcano,kbs=royal[royal['Runtime']>10],volcano[volcano['Runtime']>10],kbs[kbs['Runtime']>10]

data=pd.concat([royal,volcano,kbs])
data=data[(data['Runtime']>10)]

# data=data[(data['Institutions Code']=='KIGALI BUS SERVICES')]
# data=data[(data['Institutions Code']=='VOLCANO CITY SERVICE')]
# data=data[(data['Institutions Code']=='Royal Express')]


#format datetimes and add hour column
data['Starting Time'],data['Ending Time']=pd.to_datetime(data['Starting Time']),pd.to_datetime(data['Ending Time'])
data['Start Day'],data['End Day']=data['Starting Time'].dt.day,data['Ending Time'].dt.day
data['Hour']=data['Starting Time'].dt.hour


#calculate the frequencies and headways
frequencies=data.groupby(['Route Number','Hour','Day']).aggregate(Buses_Departing=(
    'Plate','count')).reset_index().groupby(['Route Number','Hour'])['Buses_Departing'].mean().reset_index()
frequencies['Headway']=60/frequencies['Buses_Departing']

#duplicate the frequencies and use similar freqs for both inbound and outbound directions
inbound,outbound=frequencies.copy(),frequencies.copy()
inbound['Direction'],outbound['Direction']='Inbound','Outbound'
inbound['Buses_Departing'],outbound['Buses_Departing']=inbound['Buses_Departing']/2,outbound['Buses_Departing']/2
inbound['Headway'],outbound['Headway']=60/inbound['Buses_Departing'],60/outbound['Buses_Departing']

frequencies=pd.concat([inbound,outbound])
frequencies.sort_values(by=['Route Number','Hour'],inplace=True)
frequencies.head()

,Route Number,Hour,Buses_Departing,Headway,Direction
0,101,1,1.000000,60.0,Inbound
0,101,1,1.000000,60.0,Outbound
1,101,4,0.500000,120.0,Inbound
1,101,4,0.500000,120.0,Outbound
2,101,5,0.666667,90.0,Inbound


## **Runtimes**

-- Using the taps data, drop all the trips with runtimes>3hrs and those running across multiple days  
-- Calculate the mean time it takes to complete each trip and aggregate it per route and hour of the day


In [27]:
df=data[~((data['Start Day']!=data['End Day'])|(data["Runtime"]>180))]
runtimes=df.groupby(['Route Number','Hour','Day']).aggregate(Runtime=(
    'Runtime','mean')).reset_index().groupby(['Route Number','Hour'])['Runtime'].mean().reset_index()
runtimes.head()

#add these runtimes to the frequencies
frequencies=pd.merge(frequencies,runtimes,on=['Route Number','Hour'])
frequencies['start_time']=frequencies['Hour'].apply(lambda x: datetime(2023,datetime.now().month,datetime.now().day,x,0,0))
frequencies['end_time']=frequencies['start_time']+pd.to_timedelta(frequencies['Runtime'],unit='m')

frequencies.rename(columns={
    'Route Number':'route_number',
    'Hour':'Departure_Hour',
    'Buses_Departing':'Departures per hour',
    'Headway':'headway'},inplace=True)

frequencies.head()


,route_number,Departure_Hour,Departures per hour,headway,Direction,Runtime,start_time,end_time
0,101,5,0.666667,90.000000,Inbound,55.136111,2023-09-12 05:00:00,2023-09-12 05:55:08.166666420
1,101,5,0.666667,90.000000,Outbound,55.136111,2023-09-12 05:00:00,2023-09-12 05:55:08.166666420
2,101,6,2.333333,25.714286,Inbound,36.733153,2023-09-12 06:00:00,2023-09-12 06:36:43.989197520
3,101,6,2.333333,25.714286,Outbound,36.733153,2023-09-12 06:00:00,2023-09-12 06:36:43.989197520
4,101,7,2.111111,28.421053,Inbound,40.529134,2023-09-12 07:00:00,2023-09-12 07:40:31.748015880


## Timetable

--Using the frequencies and runtimes, we develop a timetable to find the total number of trips


In [28]:
frequencies=frequencies.dropna()
start_main=[]
stop_main=[]
route_id_main=[]
direction_main=[]
for i in range(len(frequencies)):
    trip=frequencies.iloc[i:i+1]
    route_id=[]
    buses=int(math.floor(trip.iloc[0]['Departures per hour']))
    sta=[]
    sto=[]
    direction=[]
    if buses>1:
        for bus in range(buses):
            if bus==0:
                sta.append(trip.iloc[0]['start_time'])
            else:
                sta.append(sta[-1]+timedelta(minutes=trip.iloc[0]['headway']))
                
                
            route_id.append(trip.iloc[0]['route_number'])
            direction.append(trip.iloc[0]['Direction'])
#             sta.append(trip.iloc[0]['start_time'])
            sto.append(sta[-1]+timedelta(minutes=trip.iloc[0]['Runtime']))
    else:
        sta.append(trip.iloc[0]['start_time'])
        sto.append(sta[-1]+timedelta(minutes=trip.iloc[0]['Runtime']))
        route_id.append(trip.iloc[0]['route_number'])
        direction.append(trip.iloc[0]['Direction'])
    
    start_main=start_main+sta
    stop_main=stop_main+sto
    route_id_main=route_id_main+route_id
    direction_main=direction_main+direction            
    
df_timetable=pd.DataFrame(list(zip(direction_main,route_id_main,start_main,stop_main)),columns=['direction','route_id','start','stop'])
df_timetable.sort_values(by=['route_id','start'],inplace=True)



routes=df_timetable['route_id'].unique().tolist()
df_master_timetable=pd.read_csv("timetable2.csv")
df_master_timetable=df_master_timetable[df_master_timetable['Bus_Route'].isin(routes)]
df_master_timetable=df_master_timetable.drop_duplicates(subset=['Bus_Route','Direction','Departure_Terminal','Arrival_Terminal'])
df_master_timetable.rename(columns={'Bus_Route':'route_id','Direction':'direction'},inplace=True)
df_master_timetable=df_master_timetable[['direction','route_id','Departure_Terminal','Arrival_Terminal','Distance_km']]
df_master_timetable=df_master_timetable.append(pd.DataFrame({'direction':['Inbound','Outbound']*5,
                            'route_id':[149,149,150,150,156,156,'MASAKA_DOWNTOWN','MASAKA_DOWNTOWN',202,202],
    'Departure_Terminal':['Remera','12_SEZ','Rubirizi','Nyabugogo',
                         'Kanombe','Nyabugogo','Masaka','Downtown','Nyanza','Downtown'],
    'Arrival_Terminal':['12_SEZ','Remera','Nyabugogog','Rubirizi',
                       'Nyabugogo','Kanombe','Downtown','Masaka','Downtown','Nyanza'],
    'Distance_km':[14.7,14.7,17.86,17.86,21.86,21.86,23.1,23.1,10,10]}))
timetable=pd.merge(df_timetable.replace({'202*':202,'307*':307}),df_master_timetable,on=['route_id','direction']).sort_values(by=['route_id','start'])

timetable.head()

,direction,route_id,start,stop,Departure_Terminal,Arrival_Terminal,Distance_km
0,Inbound,101,2023-09-12 05:00:00.000000,2023-09-12 05:55:08.166666,Remera Taxi Park,CBD,13.832057
22,Outbound,101,2023-09-12 05:00:00.000000,2023-09-12 05:55:08.166666,CBD,Remera Taxi Park,9.676827
1,Inbound,101,2023-09-12 06:00:00.000000,2023-09-12 06:36:43.989198,Remera Taxi Park,CBD,13.832057
23,Outbound,101,2023-09-12 06:00:00.000000,2023-09-12 06:36:43.989198,CBD,Remera Taxi Park,9.676827
2,Inbound,101,2023-09-12 06:25:42.857143,2023-09-12 07:02:26.846341,Remera Taxi Park,CBD,13.832057


## TRIP ASSIGNMENT

--From the timetable, we allocate the trips to buses  
-- The aim is to have the fewest number number of buses to cover all the trips-effectively reducing the waiting time per bus on each trip  
--We make the assumption that each bus waits at least 5 minutes at the terminus  


General procedure followed:  
1.The first bus on each route chooses the shortest route direction - either inbound and outbound- as it's first trip. The trip in the opposite direction is  
  assigned to the second bus.  

2.If the next trips in the timetable begin when the first 2 buses are yet to complete their trips, a new pair of buses are assigned them and this goes on till all the trips have been alocated  

3.Buses also have to wait for at least 5 minutes after arriving at a terminus before they can be assigned to trips

In [29]:
new=timetable
new.rename(columns={'direction':'Direction','route_id':'Bus_Route','start':'Departure_Time','stop':'Arrival_Time'},inplace=True)
new['hour']=new['Departure_Time'].dt.hour
new=pd.merge(new,runtimes.rename(columns={'Route Number': 'Bus_Route','Hour':'hour'}).replace({'202*':202}),on=['Bus_Route','hour'])
new.rename(columns={'Runtime':'Runtime_mins'},inplace=True)

df_x=pd.DataFrame()  # to contain the assigned trps
unassigned=pd.DataFrame()
for rt in new['Bus_Route'].unique().tolist():
    try:
        df_bus_route=new[new.Bus_Route==rt] #selected route to run greedy algorithm
        x=pd.DataFrame()
        # for bus in range(6): # select an arbitrary number of buses to allocate trips to
        bus=1
        while len(df_bus_route)>1:
            Direction=[]
            trips_df=pd.DataFrame()
            i=0 #first trip
            t=df_bus_route['Arrival_Time'].min() #earliest the trip ends
            next_trip=['Inbound']                #if the first trip is inbound, the next one is automatically outbound
            y=df_bus_route['Direction'].unique().tolist()    #contains the Inbound and outbound directions
            while t<datetime(2023,9,datetime.now().day,23,59) and (next_trip[-1] in y): #while the arrival time is <midnight- so there is a possibility of a new trip
        #     for j in range(8):     
                    if i==0:
                        potential_trips=df_bus_route[df_bus_route['Departure_Time']==df_bus_route['Departure_Time'].min()]
                        selected_trip=potential_trips[potential_trips['Distance_km']==potential_trips['Distance_km'].min()] #select the shortest distance route
                        Direction.append(selected_trip.iloc[0]['Direction']) #obtain route direction of selected trip
                        Bus_Route=selected_trip.iloc[0]['Bus_Route']         #obtain route of selected trip
                        new_set_of_trips=df_bus_route[df_bus_route['Departure_Time']>=selected_trip.iloc[0]['Arrival_Time']+timedelta(minutes=5)]
                        trips_df=trips_df.append(selected_trip)  #add this selected/assigned trip to trips_df
                        i=i+1                                    #increment counter as we are no longer in the first trip
                    else:                                        #for the subsequent trips
                        potential_trips=new_set_of_trips[new_set_of_trips['Direction']!=Direction[-1]] #direction opposite the previous one
                        selected_trip=potential_trips[potential_trips['Departure_Time']==potential_trips['Departure_Time'].min()] #next available trip selected
                        Direction.append(selected_trip.iloc[0]['Direction'])#obtain route direction of selected trip
                        Bus_Route=selected_trip.iloc[0]['Bus_Route']        #obtain route of selected trip
                        new_set_of_trips=df_bus_route[df_bus_route['Departure_Time']>=selected_trip.iloc[0]['Arrival_Time']+timedelta(minutes=5)]
                        trips_df=trips_df.append(selected_trip) ##add this selected/assigned trip to trips_df
                        t=selected_trip.iloc[0]['Arrival_Time']  #update t which is the time when the current trip ends
                        i=i+1                                    #increment counter
                        if selected_trip.iloc[0]['Direction']=='Inbound':   #if the selected trip was an inbound one,
                            next_trip[-1]='Outbound'                        #next one has to be outbound and vice versa
                        else:
                            next_trip[-1]='Inbound'
                        y=new_set_of_trips['Direction'].to_list()          #list of directions in the new_set of trips
            df_bus_route=df_bus_route[~(df_bus_route.index.isin(trips_df.index))]
            trips_df['bus']=bus
            x=x.append(trips_df)
            bus=bus+1
                   
    except:
        pass
    u=pd.DataFrame()
    if len(df_bus_route)>0:
        v=bus-1
        for unassigned_trip in range(len(df_bus_route)):
            extra_trip=df_bus_route.iloc[unassigned_trip:unassigned_trip+1]
            extra_trip['bus']=v+1
            u=u.append(extra_trip)
            v=v+1
            
            
    df_x=df_x.append(x)
    df_x=df_x.append(u)
    unassigned=unassigned.append(df_bus_route)
    
    
df_x.head()

,Direction,Bus_Route,Departure_Time,Arrival_Time,Departure_Terminal,Arrival_Terminal,Distance_km,hour,Runtime_mins,bus
1,Outbound,101,2023-09-12 05:00:00.000000,2023-09-12 05:55:08.166666,CBD,Remera Taxi Park,9.676827,5,55.136111,1
4,Inbound,101,2023-09-12 06:25:42.857143,2023-09-12 07:02:26.846341,Remera Taxi Park,CBD,13.832057,6,36.733153,1
9,Outbound,101,2023-09-12 07:28:25.263158,2023-09-12 08:08:57.011174,CBD,Remera Taxi Park,9.676827,7,40.529134,1
12,Inbound,101,2023-09-12 08:26:20.487805,2023-09-12 09:19:22.187805,Remera Taxi Park,CBD,13.832057,8,53.028333,1
17,Outbound,101,2023-09-12 10:00:00.000000,2023-09-12 11:00:12.407407,CBD,Remera Taxi Park,9.676827,10,60.206790,1


## Summary from the trips assignment

In [30]:
route_distances=df_x[['Bus_Route','Distance_km']]
df_x_1=df_x[df_x['Bus_Route'].isin(route_distances['Bus_Route'].unique().tolist())]
df_trip_assignment=df_x_1.groupby(['Bus_Route','bus']).aggregate(Distance=('Distance_km','sum'),
                                           Trips=('bus','count')).reset_index()
df_trip_assignment['Distance_per_Trip']=df_trip_assignment['Distance']/df_trip_assignment['Trips']
df_trip_assignment

,Bus_Route,bus,Distance,Trips,Distance_per_Trip
0,101,1,141.053304,12,11.754442
1,101,2,141.053304,12,11.754442
2,101,3,103.712363,9,11.523596
3,101,4,107.867593,9,11.985288
4,101,5,13.832057,1,13.832057
...,...,...,...,...,...
144,306,3,5.524089,1,5.524089
145,306,4,5.723076,1,5.723076
146,MASAKA_DOWNTOWN,1,231.000000,10,23.100000
147,MASAKA_DOWNTOWN,2,207.900000,9,23.100000


In [31]:
import plotly.express as px
def calculate_differences(group_df):
    group_df['waiting_time'] = (group_df['Departure_Time'].shift(-1) - group_df['Arrival_Time']).dt.seconds/60
    group_df['Departure_Time']= group_df['Arrival_Time']
    group_df['Arrival_Time']=group_df['Departure_Time']+pd.to_timedelta(group_df['waiting_time'],unit='m')
    return group_df

# Apply the function within each group
result_df = df_x.groupby(['Bus_Route','bus']).apply(calculate_differences)
result_df=result_df[[
    'Direction','Bus_Route','Departure_Time','Arrival_Time','Departure_Terminal','Arrival_Terminal','Distance_km','hour','Runtime_mins','bus'
]]
result_df['status']='waiting'
df_x['status']='driving'
df_driving_waiting=pd.concat([df_x,result_df]).sort_values(by=['Bus_Route','bus','Departure_Time'])

route_of_interest=101
px.timeline(df_driving_waiting[df_driving_waiting['Bus_Route']==route_of_interest],
            x_start='Departure_Time',
            x_end='Arrival_Time',
            y='bus',
            color='status',hover_data=['Direction'],
            color_discrete_map=({'driving':'green','waiting':'goldenrod'}),
           title=f"Route {route_of_interest}")

Drop the buses doing few trips <3 from the projected results  

This is to avoid significantly bringing down the mean of the mileage/revenue  

In the above case for route 101, we would drop buses 5 and 6 as they only complete 1 trip each

In [32]:
df_trip_assignment=df_trip_assignment[df_trip_assignment['Trips']>3]
df_trip_assignment.head()

,Bus_Route,bus,Distance,Trips,Distance_per_Trip
0,101,1,141.053304,12,11.754442
1,101,2,141.053304,12,11.754442
2,101,3,103.712363,9,11.523596
3,101,4,107.867593,9,11.985288
6,102,1,300.060312,12,25.005026


#### Using the route fares, calculate the total fare that would be collected for each bus depending on the number of trips

In [33]:
ac_group_fares=pd.read_csv('Kigali Routes Data - Updated Routes - from AC Gp Data.csv')
df_trip_assignment=pd.merge(df_trip_assignment,ac_group_fares,on='Bus_Route',how='left')
df_trip_assignment.replace(df_trip_assignment.iloc[-1]['Fare(Frw)'],253,inplace=True)
df_trip_assignment['Total Revenue']=df_trip_assignment['Trips']*df_trip_assignment['Fare(Frw)']*70
df_trip_assignment

,Bus_Route,bus,Distance,Trips,Distance_per_Trip,Long line name,Fare(Frw),Total Revenue
0,101,1,141.053304,12,11.754442,REMERA _ VILLE,220.0,184800.0
1,101,2,141.053304,12,11.754442,REMERA _ VILLE,220.0,184800.0
2,101,3,103.712363,9,11.523596,REMERA _ VILLE,220.0,138600.0
3,101,4,107.867593,9,11.985288,REMERA _ VILLE,220.0,138600.0
4,102,1,300.060312,12,25.005026,KABUGA _ NYABUGOGO,517.0,434280.0
...,...,...,...,...,...,...,...,...
116,302,2,88.828234,8,11.103529,KIMIRONKO-KIE-CHEZ LANDO-CBD,253.0,141680.0
117,306,1,44.988660,8,5.623582,KIMIRONKO-ZINDIRO -MASIZI-BIREMBO,216.0,120960.0
118,306,2,44.988660,8,5.623582,KIMIRONKO-ZINDIRO -MASIZI-BIREMBO,216.0,120960.0
119,MASAKA_DOWNTOWN,1,231.000000,10,23.100000,253,253.0,177100.0


# ANALYSIS

## 1.What's the total trips?

### a) Actual Data

In [34]:
avg_daily_trips_per_operator=data.groupby(['Institutions Code','Day']).aggregate(Daily_Trips=(
    'Route Number','count')).reset_index().groupby('Institutions Code')['Daily_Trips'].mean().reset_index()
print(f"Total Daily Trips from AC group data is: {avg_daily_trips_per_operator['Daily_Trips'].sum()} trips")
avg_daily_trips_per_operator

Total Daily Trips from AC group data is: 1250.25 trips


,Institutions Code,Daily_Trips
0,KIGALI BUS SERVICES,455.25
1,Royal Express,509.60
2,VOLCANO CITY SERVICE,285.40


### b) Estimated Data

From the estimated data, the total number of trips per day is

In [35]:
df_trip_assignment['Trips'].sum()

1161

# 2. What's the total number of buses covering these routes?

### a) Actual Data

In [36]:
# 1. Unique buses
unique_buses_per_operator=data.drop_duplicates(subset=['Plate']).groupby('Institutions Code')['Plate'].count().reset_index()

print(f"the total buses per operator is {unique_buses_per_operator['Plate'].sum()} ")
unique_buses_per_operator

the total buses per operator is 112 


,Institutions Code,Plate
0,KIGALI BUS SERVICES,46
1,Royal Express,37
2,VOLCANO CITY SERVICE,29


### b) Estimated Data

In [37]:
df_trip_assignment.groupby('Bus_Route')['bus'].max().sum()

121

## 3. How does revenue compare between the actual taps data and the projected data?

### a) Actual Data

In [38]:
daily_revenue_per_bus=data.groupby(['Plate','Day'])['Total Amount'].sum().reset_index()
avg_daily_revenue_per_bus=daily_revenue_per_bus.groupby('Plate')['Total Amount'].mean().reset_index()

fig=px.histogram(avg_daily_revenue_per_bus,x='Total Amount',title='AC Group Revenue Data Distribution')
fig.add_annotation(x=380000,y=11,
                   text=f"Mean:{round(avg_daily_revenue_per_bus['Total Amount'].mean(),2)}",
                  showarrow=False,
                   font=dict(size=18))

fig.add_annotation(x=380000,y=8,
                   text=f"SD:{round(avg_daily_revenue_per_bus['Total Amount'].std(),2)}",
                  showarrow=False,
                   font=dict(size=18))


![My Plot](ac group-revenue distribution.png)


### b)Estimated data

In [39]:
df_trip_assignment1=df_trip_assignment
fig=px.histogram(df_trip_assignment1,x='Total Revenue',title='Projected RRevenue Data Distribution')


fig.add_annotation(x=380000,y=11,
                   text=f"Mean:{round(df_trip_assignment1['Total Revenue'].mean(),2)}",
                  showarrow=False,
                   font=dict(size=18))

fig.add_annotation(x=380000,y=8,
                   text=f"SD:{round(df_trip_assignment1['Total Revenue'].std(),2)}",
                  showarrow=False,
                   font=dict(size=18))


## 4.How does mileage compare between the projections and the actual taps data?

### a)Actual Data

In [40]:
data_distances=pd.merge(data.rename(columns={'Route Number':'Bus_Route'}),route_distances.drop_duplicates(subset=['Bus_Route']),on='Bus_Route')


daily_distances_per_bus=data_distances.groupby(['Plate','Day'])['Distance_km'].sum().reset_index()
avg_daily_distance_per_bus=daily_distances_per_bus.groupby('Plate')['Distance_km'].mean().reset_index()

fig=px.histogram(avg_daily_distance_per_bus,x='Distance_km',title='AC Group-Mileage Distribution')
fig.add_annotation(x=350,y=19,
                   text=f"Mean:{round(avg_daily_distance_per_bus['Distance_km'].mean(),2)}",
                  showarrow=False,
                   font=dict(size=18))

fig.add_annotation(x=350,y=16,
                   text=f"SD:{round(avg_daily_distance_per_bus['Distance_km'].std(),2)}",
                  showarrow=False,
                   font=dict(size=18))
fig.show()

### b) Estimated data

In [42]:
# df_trip_assignment1=df_trip_assignment[df_trip_assignment['Distance']>=50]
df_trip_assignment1=df_trip_assignment
fig=px.histogram(df_trip_assignment1,x='Distance',title='Estimated Data Mileage Distribution')


fig.add_annotation(x=200,y=19,
                   text=f"Mean:{round(df_trip_assignment1['Distance'].mean(),2)}",
                  showarrow=False,
                   font=dict(size=18))

fig.add_annotation(x=200,y=16,
                   text=f"SD:{round(df_trip_assignment1['Distance'].std(),2)}",
                  showarrow=False,
                   font=dict(size=18))


# 5.Trips

## On average, how many trips does a bus make from both the ac group tap data and the projected data

In [43]:
print(f"AC Group taps data: {round(data.groupby(['Plate','Day'])['Line Code'].count().reset_index()['Line Code'].mean(),1)} trips")

AC Group taps data: 13.6 trips


In [44]:
print(f"Estimated number of trips per bus: {round(df_trip_assignment[df_trip_assignment['Distance']>=70]['Trips'].mean(),1)}")

Estimated number of trips per bus: 10.6


## From the projections, what would be the number of trips on every route?

In [45]:
fig=px.bar(df_trip_assignment.groupby('Bus_Route')['Trips'].sum().reset_index(),
       x=df_trip_assignment.groupby('Bus_Route')['Trips'].sum().reset_index()['Bus_Route'].astype(str),
       y='Trips',
          text='Trips',
          title='Trips Per Route')

fig.update_layout(xaxis={'categoryorder':'total descending'},title_x=0.15,title_y=0.88)

## What would be the required number of buses on each route?

In [46]:
fig=px.bar(df_trip_assignment.groupby('Bus_Route')['bus'].max().reset_index(),
           x=df_trip_assignment.groupby('Bus_Route')['bus'].max().reset_index()['Bus_Route'].astype(str),
           y='bus',
          text='bus',
           title='Buses per Route')

fig.update_layout(xaxis={'categoryorder':'total descending'},title_y=0.88,title_x=0.35)
